Copyright 2021 David Bazzett  
The purpose of this code is to  
1) BEGIN LOOP: Generate forecast tables for NWM reanalysis timeseries data (converted from HDF5 to NetCDF format) using the NFIE hydrotables  
2) Find the maximum flows in each COMID during the analysis period and generate a single forecast table of maximum inundation data  
3) Generate an inundation map from the MaxFlow forecast  
4) Validate the inundation data against the validation data (high water marks)  
5) Update the Manning's N in the hydrotables based on the model accuracy OR EXIT LOOP  
Repeat steps 1-5 until the Manning's N is accurate and an exit condition is met

In [2]:
import pandas as pd
import xarray as xr
import dbfread as dbf
import glob

In [1]:
def UpdateManning():
    # Load data sets
    nhd = pd.DataFrame(iter(dbf.read(nhd_file)))
    hydrotable = pd.read_csv(hydrotable_file)

    # Read the unique COMIDs from your hydrotable, put them in a new dataframe
    # Add the NDH data for these COMIDS (NHD file contains data for all 2.7M COMIDS)
    df1 = pd.DataFrame(hydrotable["CatchId"].unique(), columns=["ComID"])
    df1['Order'] = ""
    df1 = df1.merge(nhd,on='ComID')

    # Create a small dataframe with just the COMID and stream order
    # Format column names and merge to hydrotable
    df2 = df1[['StreamOrde','ComID']].copy()
    df2 = df2.rename(columns={'ComID':'CatchId'})
    hydrotable = hydrotable.merge(df2,on='CatchId')

    # This is a csv of Manning's N based on stream order. The initial data is from the CAHABA Manning's data
    # Merge this roughness data into the hydrotable
    rough = pd.read_csv(rough_file)
    hydrotable = hydrotable.merge(rough,on='StreamOrde')

    # Calculate flow based on the new Manning's N
    hydrotable['Discharge (m3s-1)'] = hydrotable['WetArea (m2)'] * pow(hydrotable['HydraulicRadius (m)'],2.0/3) * \
        pow(hydrotable['SLOPE'],0.5) / hydrotable['rough']
    
    # TODO: Do we overwrite the existing hydrotable or create a new one?
    # Write new table to file (comment out for testing)
    # hydrotable.to_csv("C:/Users/David/G-Rutgers/020301/hydrogeo-fulltable-020301-updated.csv")
    print('UpdateManning complete!')

In [9]:
# The code should ideally create a blank dataframe and append all flow data to it
# Not sure how to do this, so I read the first file, then append all data to it
# This results in 1 duplicate timestep in the dataset, which doesn't make a difference

def MaxFlow():
    file_path = forecast_path + 'inun*csv'
    
    # Read the first file in the path to start the table, then break
    # This method seems awkward but it works for now
    for path in glob.iglob(file_path):
        date = path[-19:-8] # strip date and time from filename, use for column names
        df1 = pd.read_csv(path)
        df1 = df1.rename(columns={"H": date + "_H", "Q": date + "_Q"})
        break

    # Once you have the initial table set up, add the remaining data.
    # You will have duplicates of the first row because of this method, but it doesn't matter when looking for maximums
    for path in glob.iglob(file_path):
        date = path[-19:-8] # strip date and time from filename, use for column names
        df2 = pd.read_csv(path)
        df2 = df2.rename(columns={"H": date + "_H", "Q": date + "_Q"})
        df1 = df1.merge(df2, on='COMID')

    df1 = df1.set_index(['COMID'])

    # Create a dataframe of all flow values Q, find the maximum flows in the timeseries
    flow = df1.filter(like='Q', axis=1)
    flow['fmax'] = ''
    flow.fmax = flow.max(axis=1)

    # Create a dataframe of all height values H, find the maximum heights in the timeseries
    ht = df1.filter(like='H', axis=1)
    ht['hmax'] = ''
    ht.hmax = ht.max(axis=1)

    # Generate a dataframe of maximum flow and height values for each COMID in the time series
    # This will be used to generate the inundation map showing max flooding in each COMID during the event
    # This is not to be confused with instantaneous or "snapshot" flooding
    # Different COMIDs peak at different times - we want to show all maximums regardless of time
    maxtbl = pd.DataFrame(flow.fmax)
    maxtbl = maxtbl.merge(ht.hmax, left_index=True, right_index=True)
    maxtbl = maxtbl.rename(columns={"fmax": "Q", "hmax": "H"})

    maxtbl.to_csv(forecast_path + 'maxflow.csv')
    # flow.to_csv('Irene_flow.csv')
    print('MaxFlow complete!')

NOTES:  
1) Forecast tables - Roger, the NFIE forecast table script runs in python 2.7. Can we call python 2.7 from within python3? I have a bash script in Amarel to run forecast-table for every forecast in a folder.  

2) InunMap - Roger, this is called as a bash script in Amarel. Can we call a bash script from within python?

In [10]:
# Variables for UpdateManning
hydrotable_file = "C:/Users/David/G-Rutgers/020301/hydrogeo-fulltable-020301.csv"
nhd_file = 'C:/Users/David/G-Rutgers/NHDPlusAttributes/PlusFlowlineVAA.dbf'
rough_file = "./roughness.csv"

# Variables for MaxFlow
# forecast_path = './default/' # Path to forecast files (csv)
forecast_path = './manning/' # Path to forecast files (csv)

# UpdateManning()

## TODO: create function for forecast table? See note above

MaxFlow()

## TODO: create function to call InunMap? See note above

## TODO: examine InunMap tif. Compare to Irene shapefile.
## If flood depth is over / under estimated, adjust Manning's roughness based on stream order.
## Create new table of Manning's roughness vs stream order
## Loop until error in estimated / predidicted error is small, then exit.

<ipython-input-9-9438bbb040eb>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  flow['fmax'] = ''
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:5168: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
<ipython-input-9-9438bbb040eb>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guid

MaxFlow complete!
